In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
dataset = pd.read_csv('C:/Users/MVRAHUL/Downloads/fer2013.csv')

In [2]:
#declaring variables
width, height = 48, 48

datapoints = dataset['pixels'].tolist()

#getting features for training
X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

#getting labels for training
y = pd.get_dummies(dataset['emotion']).as_matrix()

X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

C:\Users\MVRAHUL\Anaconda3\envs\envi\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [3]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

num_features = 32
num_labels = 7
batch_size = 64
epochs = 25
width, height = 48, 48

In [4]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

#adding layers
classifier.add(Convolution2D( num_features,3,3 , activation='relu', input_shape=(width, height, 1)))#, data_format='channels_last', kernel_regularizer=l2(0.01)))
classifier.add(MaxPooling2D(pool_size = (2, 2)))


classifier.add(Convolution2D( num_features*2,3,3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

               
classifier.add(Flatten())


classifier.add(Dense(2*num_features, activation='relu'))
classifier.add(Dense(num_labels, activation='softmax'))

# Compiling the CNNadam
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Using TensorFlow backend.


C:\Users\MVRAHUL\Anaconda3\envs\envi\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(48, 48, 1...)`
  if sys.path[0] == '':


C:\Users\MVRAHUL\Anaconda3\envs\envi\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  app.launch_new_instance()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
# Fitting model to the Training set
classifier.fit(np.array(X_train), np.array(y_train),
          batch_size = batch_size,
          nb_epoch = epochs)
        #,validation_data=(np.array(X_valid), np.array(y_valid)),shuffle=True)

C:\Users\MVRAHUL\Anaconda3\envs\envi\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.



Epoch 1/25
28709/28709 [==============================] - 82s 3ms/step - loss: 0.3379 - acc: 0.8687
Epoch 2/25
28709/28709 [==============================] - 77s 3ms/step - loss: 0.2924 - acc: 0.8833
Epoch 3/25
28709/28709 [==============================] - 78s 3ms/step - loss: 0.2683 - acc: 0.8917: 0s - loss: 0.2684 - acc: 0.89
Epoch 4/25
28709/28709 [==============================] - 78s 3ms/step - loss: 0.2466 - acc: 0.9001: 4s 
Epoch 5/25
28709/28709 [==============================] - 78s 3ms/step - loss: 0.2242 - acc: 0.9093
Epoch 6/25
28709/28709 [==============================] - 78s 3ms/step - loss: 0.2007 - acc: 0.9192
Epoch 7/25
28709/28709 [==============================] - 78s 3ms/step - loss: 0.1765 - acc: 0.9302
Epoch 8/25
28709/28709 [==============================] - 78s 3ms/step - loss: 0.1535 - acc: 0.9395
Epoch 9/25
28709/28709 [==============================] - 78s 3ms/step - loss: 0.1299 - acc: 0.9508: 1s - loss: 0.1295 - a
Epoch 10/25
28709/28709 [===============

In [13]:
#saving trained model

from keras.models import model_from_json

#saving the  model to be used later
fer_json = classifier.to_json()
with open("xyz_2.json", "w") as json_file:
    json_file.write(fer_json)
classifier.save_weights("fer_2.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# #saving the  model to be used later
# fer_json = model.to_json()
# with open("xyz.json", "w") as json_file:
#     json_file.write(fer_json)
# classifier.save_weights("fer.h5")
# print("Saved model to disk")

In [1]:
from keras.models import model_from_json

# load json and create model
json_file = open('xyz_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("fer_2.h5")
print("Loaded model from disk")

Using TensorFlow backend.








Loaded model from disk


In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.models import model_from_json

# from keras.preprocessing.sequence import pad_sequences

import numpy
import os
import numpy as np
import cv2

#setting image resizing parameters
WIDTH = 48
HEIGHT = 48
x=None
y=None
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

#loading image
full_size_image = cv2.imread("sur.jpg")
gray=cv2.cvtColor(full_size_image,cv2.COLOR_RGB2GRAY)

face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face.detectMultiScale(gray, 1.3  , 10)
# print(faces)
#detecting faces
for (x, y, w, h) in faces:
    roi_gray = gray[y:y + h, x:x + w]
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
    cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
    cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (0, 255, 0), 1)
        
    #predicting the emotion
    yhat= loaded_model.predict(cropped_img)
    print(yhat)
    cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    print("Emotion: "+labels[int(np.argmax(yhat))])

    

cv2.imshow('Emotion', full_size_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[0. 0. 0. 0. 0. 1. 0.]]
Emotion: Surprise
